In [1]:
import pandas as pd
import matplotlib
import os
import numpy as np
import re

In [2]:
attacks_original = pd.read_csv("attacks.csv", encoding = "Iso.8859-1")
# Me ha dado el error "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 7: 
#invalid continuation byte" y he tenido que añadir el encoding

In [3]:
attacks = attacks_original.copy()

#copia del dataset original para poder volver

In [4]:
attacks_original.shape

(25723, 24)

# Cleaning up irrelevant columns and rows


Using dropna to drop rows with all empty cells

In [5]:
attacks.dropna(axis=0, how="all", thresh=None, subset=None, inplace=True)

In [6]:
attacks.shape


(8703, 24)

Also we will eliminate all rows that are duplicates

In [7]:
attacks = attacks.drop_duplicates()

In [8]:
attacks.shape

(6311, 24)

In [9]:
attacks.isna().sum()
#used to see how many missing values per column

Case Number                  1
Date                         9
Year                        11
Type                        13
Country                     59
Area                       464
Location                   549
Activity                   553
Name                       219
Sex                        574
Age                       2840
Injury                      37
Fatal (Y/N)                548
Time                      3363
Species                   2847
Investigator or Source      26
pdf                          9
href formula                10
href                         9
Case Number.1                9
Case Number.2                9
original order               2
Unnamed: 22               6310
Unnamed: 23               6309
dtype: int64

Dropping the unnamed columns since they do not offer relevant data

In [10]:
attacks.columns


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [11]:
attacks = attacks.drop(["Unnamed: 22", "Unnamed: 23"], axis=1)

In the columns we see that there are 3 different "Case Number" columns. Let's see if they offer relevant information.

In [12]:
attacks[["Case Number", "Case Number.1", "Case Number.2"]]

,Case Number,Case Number.1,Case Number.2
0,2018.06.25,2018.06.25,2018.06.25
1,2018.06.18,2018.06.18,2018.06.18
2,2018.06.09,2018.06.09,2018.06.09
3,2018.06.08,2018.06.08,2018.06.08
4,2018.06.04,2018.06.04,2018.06.04
...,...,...,...
6306,0,NaN,NaN
6307,0,NaN,NaN
6308,0,NaN,NaN
6309,0,NaN,NaN


We see that they offer duplicate information, therefore we drop both columns Case Number.1 and Case Number.2 

In [13]:
attacks = attacks.drop(["Case Number.1", "Case Number.2"], axis=1)

In [14]:
attacks.shape

(6311, 20)

In [15]:
attacks.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#There are still empty rows at the bottom that have "0" as the case number and therefore were ommited by the dropna
#There is also one with xx as the case number
#I will construct a column to group the rows by how many NaNs they have, in order to eliminate those with a big amount

In [17]:
attacks["nan_values"]=attacks.isna().sum(axis=1)

In [18]:
attacks["nan_values"].value_counts()

1     1516
0     1422
2     1200
3     1196
4      540
5      293
6      102
7       26
18       7
8        7
19       2
Name: nan_values, dtype: int64

We see that there are 2394 rows without at least 19 empty cells, so I will run a dropna with a threshold of 18 to eliminate the rows

In [19]:
attacks = attacks.dropna(axis=0, how="any", thresh=18, subset=None, inplace=False)

In [20]:
#Checking to see the end of the dataset
attacks.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,nan_values
6290,ND.0012,Before 19-Jul-1913,0.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Wading,a young Scotsman,M,...,"FATAL, leg stripped of flesh",Y,NaN,NaN,"Rural New Yorker, 7/19/1913",ND-0012-Durban-Scotsman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,13.0,3
6296,ND.0006,Before 1906,0.0,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,...,FATAL,Y,NaN,Said to involve a grey nurse shark that leapt ...,"L. Becke in New York Sun, 9/9/1906; L. Schultz...",ND-0006-ArabBoy-Prymount.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,7.0,2
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6.0,3
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4.0,3
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,...,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2.0,2


Now I drop the added column

In [21]:
attacks = attacks.drop(["nan_values"], axis=1)

In [22]:
attacks.shape

(5334, 20)

## Column Analysis

First of all I will transform the column names to easier names, eliminating caps and spaces

In [23]:
attacks.columns = attacks.columns.str.strip().str.lower().str.replace(' ', '_')

In [24]:
attacks.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'original_order'],
      dtype='object')

After looking at the columns individually, we see that we can erase the pdf and href_formula columns since they offer the same info as the href. Also the investigator_or_source can be found in the link described in href, and will not serve any statistical purposes for my hypothesis. Finally the original order does not describe anything.

In [25]:
attacks = attacks.drop(columns=["href_formula","pdf","original_order","investigator_or_source"])

In [26]:
attacks.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'href'],
      dtype='object')

In [28]:
attacks.country.value_counts()

USA                 2111
AUSTRALIA           1186
SOUTH AFRICA         526
NEW ZEALAND          117
PAPUA NEW GUINEA     104
                    ... 
IRELAND                1
AZORES                 1
PACIFIC OCEAN          1
GRAND CAYMAN           1
NICARAGUA              1
Name: country, Length: 165, dtype: int64

In [31]:
pd.set_option('display.max_rows', None)

In [32]:
attacks.type.value_counts()

Unprovoked      4071
Provoked         519
Invalid          378
Boating          148
Sea Disaster     110
Boat             104
Questionable       2
Boatomg            1
Name: type, dtype: int64